In [ ]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def get_bert_embedding(text):
    # Tokenize and encode the text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    
    # Get the hidden states (embeddings) from BERT
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the last hidden state of the [CLS] token (which represents the sentence embedding)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    
    return cls_embedding

def bert_cosine_similarity(text1, text2):
    # Get the embeddings for both texts
    embedding1 = get_bert_embedding(text1)
    embedding2 = get_bert_embedding(text2)
    
    # Compute cosine similarity
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    
    return similarity

def avg_similarity(texts):
    pairwise_sims = []
    for i, t1 in enumerate(texts):
        for t2 in texts[i:]:
            pairwise_sims.append(
                bert_cosine_similarity(t1, t2)
            )
    return sum(pairwise_sims) / len(pairwise_sims)

In [2]:
import json
data = []
with open("logs/results/9127831.ndjson", "r") as f:
    for line in f:
        data.append(json.loads(line))



In [9]:
prompts = list(filter(lambda p: p["type"] == "prompt", data))


In [10]:
scorings = list(filter(lambda p: p["type"] == "score", data))

In [12]:
usages = list(filter(lambda p: p["type"] == "usage", data))

In [11]:
len(prompts)

120

In [13]:
gens = set()
for p in prompts:
    gens.add(p["generation"])
gens

{1, 2, 3, 4, 5, 6}

In [6]:
args = data[0]
pop = args["initial_population_size"]
steps = args["max_iters"]

In [7]:
print(f"({pop=})*({steps=})={pop*steps}")

(pop=12)*(steps=10)=120


In [14]:
by_gen = []
for g in sorted(list(gens)):
    by_gen.append(list(filter(
        lambda p: p["generation"] == g, prompts
    )))

In [18]:
[len(g) for g in by_gen]

[21, 27, 31, 20, 18, 3]

In [20]:
avgs_by_gen = []
for g in by_gen:
    scores = [p["avg_fitness"] for p in g]
    avgs_by_gen.append(sum(scores)/len(scores))
avgs_by_gen

[0.6130952380952381,
 0.7222222222222222,
 0.7580645161290323,
 0.8,
 0.875,
 0.7916666666666666]

In [23]:
avg_similarity_by_gen = []
for g in by_gen:
    promps_in_gen = [p["traits"] for p in g]
    avg_similarity_by_gen.append(avg_similarity(promps_in_gen))

In [24]:
avg_similarity_by_gen

[0.901394966876868,
 0.890976014749083,
 0.8778709718777288,
 0.8909011196522486,
 0.9343709032437955,
 0.9125626881917318]